安装OCR工具

In [ ]:
pip uninstall numpy

In [1]:
pip install pytesseract

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "D:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "D:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\Anaconda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "D:\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "D:\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 205, in star

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
pip install pdfplumber pytesseract pdf2image opencv-python pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


# PDF格式的发票提取

In [12]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import cv2
#import numpy as np
from PIL import Image
import io
import os
import pandas as pd
from scipy import ndimage
from typing import List

In [13]:
def test_ocr_languages():
    """测试OCR语言包是否正常安装"""
    try:
        # 获取可用的语言
        langs = pytesseract.get_languages()
        print("已安装的语言包:", langs)
        
        # 检查关键语言是否存在
        required_langs = ['chi_sim', 'chi_tra', 'eng']
        for lang in required_langs:
            if lang in langs:
                print(f"✓ {lang} 已安装")
            else:
                print(f"✗ {lang} 未安装")
                
        return all(lang in langs for lang in required_langs)
    
    except Exception as e:
        print(f"测试语言包失败: {e}")
        return False

# 运行测试
if test_ocr_languages():
    print("语言包安装正常")
else:
    print("请检查语言包安装")

已安装的语言包: ['afr', 'amh', 'ara', 'asm', 'aze', 'aze_cyrl', 'bel', 'ben', 'bod', 'bos', 'bre', 'bul', 'cat', 'ceb', 'ces', 'chi_sim', 'chi_sim_vert', 'chi_tra', 'chi_tra_vert', 'chr', 'cos', 'cym', 'dan', 'deu', 'deu_latf', 'div', 'dzo', 'ell', 'eng', 'enm', 'epo', 'equ', 'est', 'eus', 'fao', 'fas', 'fil', 'fin', 'fra', 'frm', 'fry', 'gla', 'gle', 'glg', 'grc', 'guj', 'hat', 'heb', 'hin', 'hrv', 'hun', 'hye', 'iku', 'ind', 'isl', 'ita', 'ita_old', 'jav', 'jpn', 'jpn_vert', 'kan', 'kat', 'kat_old', 'kaz', 'khm', 'kir', 'kmr', 'kor', 'lao', 'lat', 'lav', 'lit', 'ltz', 'mal', 'mar', 'mkd', 'mlt', 'mon', 'mri', 'msa', 'mya', 'nep', 'nld', 'nor', 'oci', 'ori', 'osd', 'pan', 'pol', 'por', 'pus', 'que', 'ron', 'rus', 'san', 'sin', 'slk', 'slv', 'snd', 'spa', 'spa_old', 'sqi', 'srp', 'srp_latn', 'sun', 'swa', 'swe', 'syr', 'tam', 'tat', 'tel', 'tgk', 'tha', 'tir', 'ton', 'tur', 'uig', 'ukr', 'urd', 'uzb', 'uzb_cyrl', 'vie', 'yid', 'yor']
✓ chi_sim 已安装
✓ chi_tra 已安装
✓ eng 已安装
语言包安装正常


In [14]:
class BusinessReceptionContentValidator:
    def __init__(self):
        # 敏感词清单 - 稽核提示
        self.audit_keywords = [
            "办公用品", "配件", "耗材", "宣传品", "打印", "清洁用品", 
            "印刷品", "设备", "安装材料", "消费卡", "充值", "预付", 
            "补贴", "补助"
        ]
        
        # 敏感词清单 - 直接拒绝
        self.reject_keywords = {
            "地点": ["会所", "俱乐部", "足浴", "KTV", "保健", "健身", 
                   "农家乐", "休闲", "娱乐", "旅游"],
            "景点": ["八达岭", "十三陵", "承德避暑山庄", "五台山", "太湖", 
                   "普陀山", "黄山", "九华山", "武夷山", "庐山", "泰山", 
                   "嵩山", "武当山", "武陵源", "张家界", "白云山", 
                   "桂林漓江", "三亚热带海滨", "峨眉山", "乐山大佛", 
                   "九寨沟", "黄龙", "黄果树", "西双版纳", "华山"],
            "物品": ["香烟", "高档酒水", "鱼翅", "燕窝", "高档菜肴", 
                   "贺年卡", "明信片", "年历", "烟花爆竹", "花卉", 
                   "纪念品", "礼品"]
        }
        
        # 禁止菜肴
        self.forbidden_dishes = ["野生保护动物", "鱼翅", "燕窝"]
        
        # 酒水相关关键词
        self.alcohol_keywords = ["酒", "白酒", "红酒", "葡萄酒", "啤酒", "洋酒"]
        
    
    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """
        从PDF文件中提取文本内容，包括图片中的文字
        """
        try:
            text = ""
            
            # 1. 首先使用pdfplumber提取常规文本
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                    
                    # 2. 提取页面中的图片并进行OCR
                    image_text = self._extract_text_from_pdf_images(page, pdf_path)
                    if image_text:
                        text += "[图片文字] " + image_text + "\n"
            
            return text.strip()
        
        except Exception as e:
            print(f"提取PDF文本失败 {pdf_path}: {e}")
            return ""
    
    def _extract_text_from_pdf_images(self, page, pdf_path: str) -> str:
        """
        从PDF页面的图片中提取文字
        """
        image_text = ""
        
        try:
            # 方法1: 使用pdfplumber提取图片
            if page.images:
                for img in page.images:
                    # 这里可以进一步处理图片，但pdfplumber的图片提取功能有限
                    pass
            
            # 方法2: 使用pdf2image将页面转换为图片，然后进行OCR
            # 获取页面编号（从0开始）
            page_index = page.page_number - 1
            
            # 将特定页面转换为图片
            images = convert_from_path(
                pdf_path, 
                first_page=page.page_number, 
                last_page=page.page_number,
                dpi=1000,  # 提高DPI以获得更好的OCR效果
                poppler_path=r'C:\Users\py0040\poppler\Release-25.07.0-0\poppler-25.07.0\Library\bin'
            )
            
            if images:
                # 对每个图片进行OCR
                for img in images:
                    # 将PIL图像转换为OpenCV格式（numpy数组）
                    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                    
                    # 图像预处理以提高OCR准确率
                    processed_img = self._preprocess_image_for_ocr(img_cv)
                    
                    # 使用pytesseract进行OCR
                    #page_img_text = pytesseract.image_to_string(
                    #    processed_img, 
                    #    lang='chi_sim+eng',  # 中文简体+英文
                    #    config='--psm 6 --oem 3'  # 添加OCR引擎模式
                    #)
                    
                    #if page_img_text.strip():
                    #    image_text += page_img_text + "\n"

                        # 尝试不同的OCR配置
                    ocr_configs = [
                        '--psm 3 --oem 3',  # 完全自动的页面分割，但没有OSD
                        '--psm 6 --oem 3',  # 统一的文本块
                        '--psm 4 --oem 3',  # 假设有一列文本
                    ]
                    
                    for config in ocr_configs:
                        try:
                            page_img_text = pytesseract.image_to_string(
                                processed_img, 
                                lang='chi_sim+chi_tra+eng',
                                config=config
                            )
                            if page_img_text.strip():
                                image_text += page_img_text + "\n"
                                break  # 如果这个配置有结果，就使用它
                        except Exception as e:
                            print(f"OCR配置 {config} 失败: {e}")
                            continue
        
        except Exception as e:
            print(f"提取PDF图片文字失败: {e}")
        
        return image_text.strip()
    
    def _preprocess_image_for_ocr(self, image):
        """
        对图像进行预处理以提高OCR准确率
        """
        try:
            # 1.转换为灰度图
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image
                
            # 2. 优化对比度 CLAHE算法
            gray = self._enhance_contrast(gray)
            
            # 3. 摆正图像（倾斜校正）
            #gray = self._deskew_image(gray)

            # 4. 去噪
            # 可以使用高斯模糊去噪（这里使用中值滤波去噪）
            #blurred = cv2.GaussianBlur(gray, (5, 5), 0)

            # 应用中值滤波去噪（对中文文字效果更好）
            blurred = cv2.medianBlur(gray, 3)
            
            # 5.二值化处理
                # 方法1: Otsu's 二值化 #将图像转换为黑白两色，可以有效减少色彩和阴影的干扰，便于字符分割
            _, thresh_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                # 方法2: 自适应阈值（自适应阈值算法对光照不均的图像效果更好）
            thresh_adaptive = cv2.adaptiveThreshold(
                blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                cv2.THRESH_BINARY, 11, 2
            )

            # 可以根据需要选择其中一种方法(这里选择方法1)
            processed = thresh_otsu
            
            # 6.形态学操作去除噪声（对中文文字要小心使用，避免破坏文字结构）
            kernel = np.ones((1, 1), np.uint8)
            #processed = cv2.morphologyEx(thresh_otsu, cv2.MORPH_CLOSE, kernel)
            processed = cv2.morphologyEx(processed, cv2.MORPH_OPEN, kernel)
            
            return processed
        
        except Exception as e:
            print(f"图像预处理失败: {e}")
            return image

    
    def _enhance_contrast(self, image):
        """
        优化图像对比度
        """
        try:
            # 方法1: 直方图均衡化
            equalized = cv2.equalizeHist(image)
            
            # 方法2: CLAHE (对比度受限的自适应直方图均衡化) - 对发票效果更好
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            clahe_applied = clahe.apply(image)
            
            # 方法3: 伽马校正
            gamma = 1.5
            inv_gamma = 1.0 / gamma
            table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
            gamma_corrected = cv2.LUT(image, table)
            
            # 方法4: 非线性对比度拉伸
            # 增强暗部细节
            alpha = 1.5  # 对比度控制 (1.0-3.0)
            beta = 0     # 亮度控制 (0-100)
            contrast_stretched = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
            
            # 可以根据实际情况选择最佳方法
            # 对于发票，CLAHE通常效果最好
            return clahe_applied
            
        except Exception as e:
            print(f"对比度增强失败: {e}")
            return image
    
    def _deskew_image(self, image):
        """
        摆正图像（倾斜校正）
        """
        try:
            # 方法1: 使用霍夫变换检测直线并计算倾斜角度
            def compute_skew_angle(image):
                # 边缘检测
                edges = cv2.Canny(image, 50, 150, apertureSize=3)
                
                # 霍夫线变换
                lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
                
                if lines is None:
                    return 0
                    
                angles = []
                for rho, theta in lines[:, 0]:
                    angle = theta * 180 / np.pi - 90
                    # 只考虑接近水平或垂直的线
                    if abs(angle) < 45:
                        angles.append(angle)
                
                if not angles:
                    return 0
                    
                # 使用中位数减少异常值影响
                median_angle = np.median(angles)
                return median_angle
            
            # 方法2: 使用最小外接矩形计算倾斜角度
            def compute_skew_angle_min_area_rect(image):
                # 二值化
                _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                
                # 查找轮廓
                contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if not contours:
                    return 0
                    
                # 找到最大轮廓
                largest_contour = max(contours, key=cv2.contourArea)
                
                # 计算最小外接矩形
                rect = cv2.minAreaRect(largest_contour)
                angle = rect[2]
                
                # 调整角度范围
                if angle < -45:
                    angle = 90 + angle
                elif angle > 45:
                    angle = angle - 90
                    
                return angle
            
            # 方法3: 使用投影轮廓分析
            def compute_skew_angle_projection(image):
                # 二值化
                _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                
                # 尝试不同角度并计算投影方差
                best_angle = 0
                max_variance = 0
                
                for angle in np.arange(-10, 10, 0.5):
                    rotated = ndimage.rotate(binary, angle, reshape=False, cval=1)
                    projection = np.sum(rotated, axis=1)
                    variance = np.var(projection)
                    
                    if variance > max_variance:
                        max_variance = variance
                        best_angle = angle
                        
                return best_angle
            
            # 尝试多种方法计算倾斜角度
            angle1 = compute_skew_angle(image)
            angle2 = compute_skew_angle_min_area_rect(image)
            angle3 = compute_skew_angle_projection(image)
            
            # 选择最可能的倾斜角度（排除异常值）
            angles = [angle1, angle2, angle3]
            angles = [a for a in angles if abs(a) < 10]  # 假设倾斜角度不会超过10度
            
            if not angles:
                return image
                
            final_angle = np.median(angles)
            
            # 如果倾斜角度很小，不进行旋转
            if abs(final_angle) < 0.5:
                return image
                
            print(f"检测到图像倾斜角度: {final_angle:.2f}度，正在校正...")
            
            # 旋转图像
            (h, w) = image.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, final_angle, 1.0)
            deskewed = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, 
                                     borderMode=cv2.BORDER_REPLICATE)
            
            return deskewed
            
        except Exception as e:
            print(f"图像倾斜校正失败: {e}")
            return image
    
    def _remove_shadow(self, image):
        """
        去除图像阴影（对扫描文档特别有用）
        """
        try:
            # 方法1: 使用形态学操作去除阴影
            rgb_planes = cv2.split(image)
            result_planes = []
            
            for plane in rgb_planes:
                # 使用形态学开运算估计背景
                dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
                bg_img = cv2.medianBlur(dilated_img, 21)
                
                # 减去背景
                diff_img = 255 - cv2.absdiff(plane, bg_img)
                
                # 归一化
                norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, 
                                       norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
                result_planes.append(norm_img)
            
            shadow_removed = cv2.merge(result_planes)
            return shadow_removed
            
        except Exception as e:
            print(f"阴影去除失败: {e}")
            return image

    #使用OCR识别方法提取整个PDF的图片文字
    def _ocr_entire_pdf(self, pdf_path: str) -> str:
        """
        使用OCR提取整个PDF的文字
        """
        try:
            # 将PDF转换为图片列表
            images = convert_from_path(pdf_path, dpi=300, poppler_path=r'C:\Users\py0040\poppler\Release-25.07.0-0\poppler-25.07.0\Library\bin')
            
            full_text = ""
            
            for i, image in enumerate(images):
                # 图像预处理
                img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
                processed_img = self._preprocess_image_for_ocr(img_cv)
                
                # OCR识别
                page_text = pytesseract.image_to_string(
                    processed_img, 
                    lang='chi_sim+chi_tra+eng',
                    config='--psm 3 --oem 3'  
                )
                
                if page_text.strip():
                    full_text += f"第{i+1}页:\n{page_text}\n\n"
            
            return full_text.strip()
        
        except Exception as e:
            print(f"OCR提取PDF失败 {pdf_path}: {e}")
            return ""
    
    # 综合方案：分情况使用两种方法提取整个PDF的图片文字
    def extract_text_from_pdf_comprehensive(self, pdf_path: str) -> str:
        """
        综合提取PDF文字（文本+图片OCR）
        这是另一种实现方式，更简单直接
        """
        all_text = ""
        
        try:
            # 1. 先用pdfplumber提取文本
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        all_text += page_text + "\n"
            
            # 2. 如果文本内容很少，尝试用OCR提取整个PDF
            if len(all_text.strip()) < 10000000:  # 如果提取的文本很少，可能是扫描版PDF
                print(f"检测到PDF {pdf_path} 可能是扫描版，使用OCR提取文字...")
                ocr_text = self._ocr_entire_pdf(pdf_path)
                all_text += "\n[OCR结果]\n" + ocr_text
            
            return all_text.strip()
        
        except Exception as e:
            print(f"提取PDF文字失败 {pdf_path}: {e}")
            return ""



### 批量将提取的发票内容存入EXCEL中，用于自动化检查

In [15]:
def batch_extract_pdf_texts(pdf_directory: str, output_excel_path: str) -> None:
    """
    批量提取多个PDF文件的文本内容，并将三种提取方法的结果写入Excel
    
    参数:
    pdf_directory: PDF文件所在目录
    output_excel_path: 输出Excel文件路径
    """
    # 创建验证器实例
    validator = BusinessReceptionContentValidator()
    
    # 获取目录下所有PDF文件
    pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"在目录 {pdf_directory} 中未找到PDF文件")
        return
    
    print(f"找到 {len(pdf_files)} 个PDF文件，开始提取文本...")
    
    # 创建结果DataFrame
    results = []
    
    for i, pdf_file in enumerate(pdf_files, 1):
        print(f"正在处理第 {i}/{len(pdf_files)} 个文件: {pdf_file}")
        
        pdf_path = os.path.join(pdf_directory, pdf_file)
        
        # 使用三种方法提取文本
        try:
            text1 = validator.extract_text_from_pdf(pdf_path)
            text2 = validator.extract_text_from_pdf_comprehensive(pdf_path)
            text3 = validator._ocr_entire_pdf(pdf_path)
            
            # 添加到结果列表
            results.append({
                '文件名': pdf_file,
                'extract_text_from_pdf结果': text1,
                'extract_text_from_pdf_comprehensive结果': text2,
                '_ocr_entire_pdf结果': text3
            })
            
        except Exception as e:
            print(f"处理文件 {pdf_file} 时出错: {e}")
            # 即使出错也添加记录，但文本为空
            results.append({
                '文件名': pdf_file,
                'extract_text_from_pdf结果': f"提取失败: {str(e)}",
                'extract_text_from_pdf_comprehensive结果': f"提取失败: {str(e)}",
                '_ocr_entire_pdf结果': f"提取失败: {str(e)}"
            })
    
    # 创建DataFrame
    df = pd.DataFrame(results)
    
    # 保存到Excel
    try:
        df.to_excel(output_excel_path, index=False, engine='openpyxl')
        print(f"结果已保存到: {output_excel_path}")
        print(f"成功处理 {len(results)} 个文件")
        
        # 显示统计信息
        success_count = len([r for r in results if not r['extract_text_from_pdf结果'].startswith('提取失败')])
        print(f"成功提取: {success_count}/{len(pdf_files)}")
        
    except Exception as e:
        print(f"保存Excel文件失败: {e}")

def batch_extract_with_progress(pdf_directory: str, output_excel_path: str, 
                               include_file_info: bool = True) -> None:
    """
    带进度显示的批量提取函数，可选择性包含文件信息
    
    参数:
    pdf_directory: PDF文件所在目录
    output_excel_path: 输出Excel文件路径
    include_file_info: 是否包含文件大小和修改时间等信息
    """
    import time
    from datetime import datetime
    
    validator = BusinessReceptionContentValidator()
    pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"在目录 {pdf_directory} 中未找到PDF文件")
        return
    
    print(f"找到 {len(pdf_files)} 个PDF文件，开始提取文本...")
    
    results = []
    start_time = time.time()
    
    for i, pdf_file in enumerate(pdf_files, 1):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        
        # 显示进度
        progress = i / len(pdf_files) * 100
        print(f"[{progress:6.2f}%] 处理: {pdf_file}")
        
        # 获取文件信息（可选）
        file_info = {}
        if include_file_info:
            try:
                stat = os.stat(pdf_path)
                file_info = {
                    '文件大小(MB)': round(stat.st_size / (1024 * 1024), 2),
                    '修改时间': datetime.fromtimestamp(stat.st_mtime).strftime('%Y-%m-%d %H:%M:%S')
                }
            except:
                file_info = {'文件大小(MB)': 'N/A', '修改时间': 'N/A'}
        
        # 提取文本
        try:
            text1 = validator.extract_text_from_pdf(pdf_path)
            text2 = validator.extract_text_from_pdf_comprehensive(pdf_path)
            text3 = validator._ocr_entire_pdf(pdf_path)
            
            # 计算文本长度
            len1, len2, len3 = len(text1), len(text2), len(text3)
            
            result = {
                '文件名': pdf_file,
                'extract_text_from_pdf结果': text1,
                'extract_text_from_pdf_comprehensive结果': text2,
                '_ocr_entire_pdf结果': text3,
                '方法1文本长度': len1,
                '方法2文本长度': len2,
                '方法3文本长度': len3,
                '最佳方法': ['方法1', '方法2', '方法3'][[len1, len2, len3].index(max(len1, len2, len3))]
            }
            
            # 添加文件信息
            result.update(file_info)
            
            results.append(result)
            
        except Exception as e:
            print(f"  错误: {str(e)}")
            result = {
                '文件名': pdf_file,
                'extract_text_from_pdf结果': f"提取失败: {str(e)}",
                'extract_text_from_pdf_comprehensive结果': f"提取失败: {str(e)}",
                '_ocr_entire_pdf结果': f"提取失败: {str(e)}",
                '方法1文本长度': 0,
                '方法2文本长度': 0,
                '方法3文本长度': 0,
                '最佳方法': '提取失败'
            }
            result.update(file_info)
            results.append(result)
    
    # 创建DataFrame并保存
    df = pd.DataFrame(results)
    
    try:
        with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
            # 主数据表
            df.to_excel(writer, sheet_name='提取结果', index=False)
            
            # 统计信息表
            total_time = time.time() - start_time
            stats = {
                '总文件数': len(pdf_files),
                '成功提取数': len([r for r in results if not r['extract_text_from_pdf结果'].startswith('提取失败')]),
                '总处理时间(秒)': round(total_time, 2),
                '平均每个文件时间(秒)': round(total_time / len(pdf_files), 2),
                '方法1平均文本长度': df['方法1文本长度'].mean(),
                '方法2平均文本长度': df['方法2文本长度'].mean(),
                '方法3平均文本长度': df['方法3文本长度'].mean(),
                '最佳方法统计': df['最佳方法'].value_counts().to_dict()
            }
            
            stats_df = pd.DataFrame([stats])
            stats_df.to_excel(writer, sheet_name='统计信息', index=False)
        
        print(f"\n完成! 结果已保存到: {output_excel_path}")
        print(f"总耗时: {total_time:.2f} 秒")
        
    except Exception as e:
        print(f"保存Excel文件失败: {e}")


### 测试和调整发票的OCR识别效果(可以多用不同的发票调优，准确度会更高)

In [29]:
# 1. 首先，创建类的实例
validator = BusinessReceptionContentValidator()

# 2. 通过实例调用方法，此时只需传入 pdf_path 参数
text1 = validator.extract_text_from_pdf("./invoice1.pdf")
text2 = validator._ocr_entire_pdf("./invoice1.pdf")
text3 = validator.extract_text_from_pdf_comprehensive("./invoice1.pdf")

#print("提取的文字V1:", text1)

检测到PDF ./invoice1.pdf 可能是扫描版，使用OCR提取文字...


In [30]:
print("提取的文字V3:", text2) #该发票实例用OCR提取效果更好

提取的文字V3: 第1页:
(出  恒生银行
HANG SENG BANK
Powered by K11KnOgS

GF RG MER INFORNAT Jun

COBO HOUSE

Shop No. 682, Level 6, Kil

Musea, Victoria Dockside, 18

Salisbury Road, [sim Sha Tsui,
n

3                    商戶號 MID       000894727874803
          終端號 TID       62007909
他                      wo tree cee

:                         销售 SALE

二号 CAHD ru
FEEE KER dada UGG (C)    44 ft
上上面如 Canp Tyre          Up baTE/TIME
”VTSA                  11/82 13:42
WEES arTCH                HR Mages thace
000015                     6002.4
SSR REF no           BRS APP Cou
20079098001               964021
APP:                 VISA CREDIT
AID:              Ait)1000003 1018
Te:             CFOE 44 259E59467
TVR: 0900008000        TST: F800
BASE:                 $21,606 .00

$4 BRL Mf ay Bat SERRE
§ ACKMOULEDGE SATISrHL TORY RECEIPT UF RELATIVE
GOODS + SERVICES borne hot f have read aud
uderstood Lie Tags ond Comlitions FOR
IATEREST-FRLE INSTALNENE FLAN AND AGIIEE TU UE
Bulgin BY Then

X 4K AG8 iy CARDHO

In [26]:
#print("提取的文字V4:", text3)

### 将589张发票输入OCR模型提取信息。

In [ ]:
    # 基本用法
    batch_extract_pdf_texts(
        pdf_directory="./invoice_pdfs",
        output_excel_path="./pdf_text_extraction_results.xlsx"
    )

In [ ]:
    # 高级用法（带进度和统计信息）
    batch_extract_with_progress(
        pdf_directory="./invoice_pdfs",
        output_excel_path="./pdf_text_extraction_detailed.xlsx",
        include_file_info=True
    )

# 图片格式的发票检查

In [44]:
#直接使用PDF的OCR识别方法，不做文件格式转换